In [41]:
!pip install pybaseball

In [42]:
import pandas as pd
import numpy as np
import pybaseball
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, classification_report, accuracy_score, mean_squared_error, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from pybaseball import statcast
from pybaseball import playerid_lookup, statcast_pitcher




# Pitcher Slection

User selects pitcher name to then find his player id to input into the model

In [43]:
start_date = '2023-04-01'
end_date = '2023-04-10'
player_info = playerid_lookup('Olson', 'Matt')
player_id = player_info['key_mlbam'].values[0]
brasier_data = statcast_pitcher(start_date, end_date, player_id)
print(f"Player ID for Sean Manaea: {player_id}")

Gathering Player Data
Player ID for Sean Manaea: 621566


# Data Set

User inputs pitcher id and may adjust the specfied dates

In [44]:
start_date = '2023-04-01'
end_date = '2023-05-10'
data_pitch_speed = statcast(start_dt=start_date, end_dt=end_date)
selected_columns = [
    'pitch_type', 'stand', 'p_throws', 
    'balls', 'strikes', 'game_year', 'outs_when_up', 'inning', 
    'pitch_number', 'bat_score', 'effective_speed', 'pfx_x',
    'pfx_z', 'batter', 'pitcher',
    'fld_score'
]
filtered_data = data_pitch_speed[selected_columns]
effective_speed_data = filtered_data[filtered_data['pitcher'] == 640455]


This is a large query, it may take a moment to complete


  0%|          | 0/40 [00:00<?, ?it/s]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  2%|▎         | 1/40 [00:00<00:31,  1.23it/s]/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. U

In [45]:
effective_speed_data

,pitch_type,stand,p_throws,balls,strikes,game_year,outs_when_up,inning,pitch_number,bat_score,effective_speed,pfx_x,pfx_z,batter,pitcher,fld_score
463,SI,R,L,1,1,2023,2,3,3,6,92.7,1.6,0.87,545121,640455,0
492,CH,R,L,0,1,2023,2,3,2,6,86.8,1.28,0.76,545121,640455,0
530,FF,R,L,0,0,2023,2,3,1,6,91.9,1.29,1.02,545121,640455,0
537,CH,R,L,1,0,2023,2,3,2,4,86.1,1.15,0.49,656448,640455,0
583,CH,R,L,0,0,2023,2,3,1,4,87.8,1.39,0.75,656448,640455,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4015,FF,L,L,3,1,2023,0,7,5,0,97.2,1.11,0.97,643217,640455,7
4127,FF,L,L,3,0,2023,0,7,4,0,96.9,1.18,0.88,643217,640455,7
4323,FF,L,L,2,0,2023,0,7,3,0,98.3,1.16,0.86,643217,640455,7
4465,FF,L,L,1,0,2023,0,7,2,0,97.3,1.16,0.93,643217,640455,7


## Label Encoding

Encoding categorical features and converting categorical columns to numerical using Label Encoding

In [46]:
le_stand = LabelEncoder()
le_p_throws = LabelEncoder()

effective_speed_data['stand'] = le_stand.fit_transform(effective_speed_data['stand'].astype(str))
effective_speed_data['p_throws'] = le_p_throws.fit_transform(effective_speed_data['p_throws'].astype(str))

/var/folders/bc/69frl30s4xv269mkcmjkv2qr0000gn/T/ipykernel_12317/3948921907.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effective_speed_data['stand'] = le_stand.fit_transform(effective_speed_data['stand'].astype(str))
/var/folders/bc/69frl30s4xv269mkcmjkv2qr0000gn/T/ipykernel_12317/3948921907.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  effective_speed_data['p_throws'] = le_p_throws.fit_transform(effective_speed_data['p_throws'].astype(str))


## One Hot Encoding

One-Hot Encoding for pitch_type then concatenates one-hot encoded pitch types with the original DataFrame

In [47]:
ohe = OneHotEncoder(sparse_output=False)
pitch_type_ohe = ohe.fit_transform(effective_speed_data[['pitch_type']])
pitch_type_ohe_df = pd.DataFrame(pitch_type_ohe, columns=ohe.get_feature_names_out(['pitch_type']))

effective_speed_data = pd.concat([effective_speed_data.reset_index(drop=True), pitch_type_ohe_df], axis=1)

## Feature Engineering

In [48]:

effective_speed_data = effective_speed_data.drop(['pitch_type'], axis=1)

interaction_features = ['balls', 'strikes', 'outs_when_up', 'inning', 'pitch_number', 'bat_score', 'fld_score']
for feature1, feature2 in itertools.combinations(interaction_features, 2):
    interaction_term = f"{feature1}_x_{feature2}"
    effective_speed_data[interaction_term] = effective_speed_data[feature1] * effective_speed_data[feature2]

print(effective_speed_data.columns)
print(effective_speed_data)

Index(['stand', 'p_throws', 'balls', 'strikes', 'game_year', 'outs_when_up',
       'inning', 'pitch_number', 'bat_score', 'effective_speed', 'pfx_x',
       'pfx_z', 'batter', 'pitcher', 'fld_score', 'pitch_type_CH',
       'pitch_type_FF', 'pitch_type_SI', 'pitch_type_SL', 'balls_x_strikes',
       'balls_x_outs_when_up', 'balls_x_inning', 'balls_x_pitch_number',
       'balls_x_bat_score', 'balls_x_fld_score', 'strikes_x_outs_when_up',
       'strikes_x_inning', 'strikes_x_pitch_number', 'strikes_x_bat_score',
       'strikes_x_fld_score', 'outs_when_up_x_inning',
       'outs_when_up_x_pitch_number', 'outs_when_up_x_bat_score',
       'outs_when_up_x_fld_score', 'inning_x_pitch_number',
       'inning_x_bat_score', 'inning_x_fld_score', 'pitch_number_x_bat_score',
       'pitch_number_x_fld_score', 'bat_score_x_fld_score'],
      dtype='object')
     stand  p_throws  balls  strikes  game_year  outs_when_up  inning  \
0        1         0      1        1       2023             2    

Seperates regressors for each pitch type

In [49]:
pitch_type_models = {}
pitch_types = ohe.get_feature_names_out(['pitch_type'])

## Test Train Sets for Pitch Type

Filters data for the current pitch type, then splis data into test train sets to create and train the Random Forest Regressor

In [50]:
for pitch_type in pitch_types:
    pitch_type_data = effective_speed_data[effective_speed_data[pitch_type] == 1]
    X = pitch_type_data.drop(['effective_speed', 'pitcher'], axis=1)
    y = pitch_type_data['effective_speed']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_regressor.fit(X_train, y_train)
    pitch_type_models[pitch_type] = rf_regressor

Creats Random Forest Classifier for pitch type prediction

In [51]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
y_pitch = effective_speed_data[pitch_types].idxmax(axis=1)
X_train_pitch, X_test_pitch, y_train_pitch, y_test_pitch = train_test_split(effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1), y_pitch, test_size=0.2, random_state=42)
rf_classifier.fit(X_train_pitch, y_train_pitch)

RandomForestClassifier(random_state=42)

# Manual Input Values for Pitch Prediction

User is able to input all values to refelct a live prediction during the baseball game

In [52]:

input_data = pd.DataFrame([{
    'stand': le_stand.transform(['R'])[0] if 'R' in le_stand.classes_ else -1,
    'p_throws': le_p_throws.transform(['L'])[0] if 'L' in le_p_throws.classes_ else -1,
    'balls': 0,
    'strikes': 0,
    'game_year': 2023,
    'outs_when_up': 2,
    'inning': 9,
    'pitch_number': 0,
    'bat_score': 3,
    'fld_score': 3,
    'pfx_x': 0.2,
    'pfx_z': 0.2,
    'batter': 621566
    'effective_speed': 86
}])

Adds one-hot encoded pitch types with default values of 0 and interaction terms to input_data

In [53]:
for col in pitch_types:
    input_data[col] = 0

for feature1, feature2 in itertools.combinations(interaction_features, 2):
    interaction_term = f"{feature1}_x_{feature2}"
    input_data[interaction_term] = input_data[feature1] * input_data[feature2]

Ensures input_data has all columns that were used during training

In [54]:
missing_cols = set(effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1).columns) - set(input_data.columns)
for col in missing_cols:
    input_data[col] = 0
input_data = input_data[effective_speed_data.drop(['effective_speed', 'pitcher'], axis=1).columns]

# Predicts pitch type probabilities

In [55]:
pitch_type_probabilities = rf_classifier.predict_proba(input_data)[0]
pitch_type_probs = dict(zip(pitch_types, pitch_type_probabilities))

# Predicts effective speed for each pitch type (using separate models for each pitch type

In [56]:
predicted_speeds = {}
for pitch_type in pitch_types:
    input_data_copy = input_data.copy()
    input_data_copy[pitch_type] = 1 
    model = pitch_type_models[pitch_type]
    effective_speed_prediction = model.predict(input_data_copy)[0]
    predicted_speeds[pitch_type] = effective_speed_prediction

# Prediction Set Data Frame

Creats a new DataFrame for the predictions

In [57]:
def create_prediction_dataframe(pitch_type_probs, predicted_speeds):
    df = pd.DataFrame({
        'Pitch Type': list(pitch_type_probs.keys()),
        'Probability': list(pitch_type_probs.values()),
        'Predicted Effective Speed': list(predicted_speeds.values())
    })
    return df

prediction_df = create_prediction_dataframe(pitch_type_probs, predicted_speeds)

print("Prediction DataFrame:")
prediction_df



Prediction DataFrame:


,Pitch Type,Probability,Predicted Effective Speed
0,pitch_type_CH,0.25,86.031
1,pitch_type_FF,0.17,92.930
2,pitch_type_SI,0.04,94.328
3,pitch_type_SL,0.54,85.450


# Confusion Matrix for Testing Set

In [65]:
print("Confusion Matrix for Testing Set:")
conf_matrix = confusion_matrix(y_test_pitch, y_test_pitch_pred)
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix for Pitch Type Prediction')
plt.show()


Confusion Matrix for Testing Set:


ValueError: Found input variables with inconsistent numbers of samples: [107, 32]

# Cross-Validation for Regression Metrics (R², RMSE, MAE)

In [21]:
for pitch_type, model in pitch_type_models.items():
    X = effective_speed_data[effective_speed_data[pitch_type] == 1].drop(['effective_speed', 'pitcher'], axis=1)
    y = effective_speed_data[effective_speed_data[pitch_type] == 1]['effective_speed']
    
    r2_scores = cross_val_score(model, X, y, cv=3, scoring='r2')
    mean_r2 = np.mean(r2_scores)
    
    neg_mse_scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-neg_mse_scores)
    mean_rmse = np.mean(rmse_scores)
    
    neg_mae_scores = cross_val_score(model, X, y, cv=3, scoring='neg_mean_absolute_error')
    mean_mae = -np.mean(neg_mae_scores)
    
    print(f"Cross-Validation Results for {pitch_type}:")
    print(f"Mean R²: {mean_r2}")
    print(f"Mean RMSE: {mean_rmse}")
    print(f"Mean MAE: {mean_mae}")
    print("\n")

Cross-Validation Results for pitch_type_CH:
Mean R²: -0.20657027551729054
Mean RMSE: 1.6408324467062163
Mean MAE: 1.323782608695652


Cross-Validation Results for pitch_type_FF:
Mean R²: -0.44336510930577977
Mean RMSE: 1.7254936109046068
Mean MAE: 1.4557339371155165


Cross-Validation Results for pitch_type_SI:
Mean R²: -7.7190939314697005
Mean RMSE: 1.2713361089625739
Mean MAE: 1.2423333333333322


Cross-Validation Results for pitch_type_SL:
Mean R²: -0.1361857564540715
Mean RMSE: 1.990074466200232
Mean MAE: 1.6610566037735761




# Hyperparameter tuning using RandomizedSearchCV for Random Forest Regressor

Defines parameter grid for tuning

In [22]:
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']  
}

Performs RandomizedSearchCV for each pitch type model and evaluates model performance on training data

In [23]:
best_pitch_type_models = {}

for pitch_type, model in pitch_type_models.items():
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=10, cv=3, 
                                       verbose=2, random_state=42, n_jobs=-1)
    X = effective_speed_data[effective_speed_data[pitch_type] == 1].drop(['effective_speed', 'pitcher'], axis=1)
    y = effective_speed_data[effective_speed_data[pitch_type] == 1]['effective_speed']
    random_search.fit(X, y)
    best_pitch_type_models[pitch_type] = random_search.best_estimator_

    y_train_pred = random_search.predict(X)
    r2 = r2_score(y, y_train_pred)
    rmse = np.sqrt(mean_squared_error(y, y_train_pred))
    print(f"R² score for {pitch_type}: {r2}")
    print(f"RMSE for {pitch_type}: {rmse}")

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.2s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.5s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimat

Updates pitch type models with the best estimators

In [ ]:
pitch_type_models = best_pitch_type_models

# Hyperparameter tuning for Random Forest Classifier (Pitch Type Prediction)

In [73]:
param_grid_classifier = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
random_search_classifier = RandomizedSearchCV(estimator=rf_classifier, param_distributions=param_grid_classifier, 
                                              n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
random_search_classifier.fit(X_train_pitch, y_train_pitch)
rf_classifier = random_search_classifier.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s


/opt/anaconda3/envs/learn-env/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=3.
  warnings.warn(


[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=30, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.1s
[CV] END max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.2s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=150; total time=   0.3s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, max_

Evaluate classifier performance on training and testing data

In [74]:
y_train_pitch_pred = rf_classifier.predict(X_train_pitch)
y_test_pitch_pred = rf_classifier.predict(X_test_pitch)

The classification report for the testing set helps us understand how well the model generalizes to unseen data. As the metrics improved compared to the original model, it means that the hyperparameter tuning has resulted in a more effective model for predicting pitch types.

In [76]:
print("Classification report for training set after tuning:")
print(classification_report(y_train_pitch, y_train_pitch_pred, zero_division=1))

print("Classification report for testing set after tuning:")
print(classification_report(y_test_pitch, y_test_pitch_pred, zero_division=1))

Classification report for training set after tuning:
               precision    recall  f1-score   support

pitch_type_CH       1.00      1.00      1.00        33
pitch_type_FF       1.00      1.00      1.00        35
pitch_type_SI       1.00      1.00      1.00         2
pitch_type_SL       1.00      1.00      1.00        20

     accuracy                           1.00        90
    macro avg       1.00      1.00      1.00        90
 weighted avg       1.00      1.00      1.00        90

Classification report for testing set after tuning:
               precision    recall  f1-score   support

pitch_type_CH       1.00      1.00      1.00         8
pitch_type_FF       1.00      1.00      1.00         6
pitch_type_SI       1.00      1.00      1.00         1
pitch_type_SL       1.00      1.00      1.00         8

     accuracy                           1.00        23
    macro avg       1.00      1.00      1.00        23
 weighted avg       1.00      1.00      1.00        23



## Evaluating the accuracy and F1 score of predicting the next pitch type

In [78]:
accuracy_train = accuracy_score(y_train_pitch, y_train_pitch_pred)
accuracy_test = accuracy_score(y_test_pitch, y_test_pitch_pred)
f1_train = f1_score(y_train_pitch, y_train_pitch_pred, average='macro', zero_division=1)
f1_test = f1_score(y_test_pitch, y_test_pitch_pred, average='macro', zero_division=1)

In [79]:
print(f"Training accuracy for predicting the next pitch type: {accuracy_train}")
print(f"Testing accuracy for predicting the next pitch type: {accuracy_test}")
print(f"Training F1 score for predicting the next pitch type: {f1_train}")
print(f"Testing F1 score for predicting the next pitch type: {f1_test}")



Training accuracy for predicting the next pitch type: 1.0
Testing accuracy for predicting the next pitch type: 1.0
Training F1 score for predicting the next pitch type: 1.0
Testing F1 score for predicting the next pitch type: 1.0


# Conclusion


The results of the hyperparameter tuning show both promising improvements and some areas of concern. On the positive side, the tuned model's performance is exceptional in terms of accuracy, precision, recall, and F1-score, with perfect values (1.0) across the board for both the training and testing sets. This suggests that the tuning process effectively optimized the model to correctly classify pitch types in the given data, leading to an apparent flawless classification. Such high performance metrics indicate that the model is very well-suited to the current data, capturing its underlying patterns.

However, the perfect scores also highlight some significant drawbacks. The most notable issue is overfitting—when the model learns the training data so well that it loses the ability to generalize to new data. In this case, the fact that both training and testing sets yield perfect scores implies that the model may be "memorizing" the training data instead of learning generalizable relationships. Additionally, the original model's cross-validation scores for R² were negative, and RMSE and MAE values were relatively high, indicating a poor fit. While hyperparameter tuning drastically improved results, the lack of any variance between training and testing performance suggests the model might not be reliable in real-world scenarios where data may be more diverse and unpredictable.

The mean R² scores for some pitch types were negative, indicating a poor fit of the model to the validation sets and a lack of generalization capability. For example, the mean R² for pitch_type_CH was -3.20, and for pitch_type_FF, it was -2.21, indicating significant deviations from actual values during cross-validation. The Root Mean Square Error (RMSE) and Mean Absolute Error (MAE) also revealed notable prediction errors for most pitch types, reinforcing the idea that while the model fits the training and testing datasets well, it struggles with generalized predictions. This discrepancy highlights the potential overfitting of the hyperparameter-tuned model and indicates a need for more diverse data or further regularization to improve its robustness.

Balancing model complexity and performance by introducing regularization or adding more data points may help mitigate these issues, as well as completley starting from scratch.